# 📄 PDF Translator - Google Colab

**Translate scientific PDFs with local LLMs (Ollama) or OpenAI**

> 🔗 **GitHub:** [error-wtf/pdf-translator-enhanced](https://github.com/error-wtf/pdf-translator-enhanced)  
> Based on [thelanguagenerd/pdf-translator](https://github.com/thelanguagenerd/pdf-translator)

© 2025 Sven Kalinowski with small help of Lino Casu  
Licensed under the Anti-Capitalist Software License v1.4

---

## 🆕 Features

- **🔬 Formula Protection** - Mathematical formulas stay intact during translation
- **🌍 20 Languages** - German, English, French, Spanish, Arabic, Hebrew, Ukrainian, Japanese, Chinese, and more
- **🔒 100% Local** - With Ollama, no data leaves Google's servers
- **⚡ GPU Accelerated** - Uses Colab's free T4 GPU

---

## Two Options:

### Option A: Ollama (FREE, runs locally on Colab GPU)
- No API key needed
- Runs entirely on Google's servers
- Your data stays in your Colab session
- Requires GPU runtime (T4 is free)

### Option B: OpenAI API (paid, better quality)
- Requires OpenAI API key
- Better translation quality (GPT-4)
- Your data is sent to OpenAI servers

---

## GPU Runtime Setup

1. Click **Runtime** → **Change runtime type**
2. Select **T4 GPU** (free) or **A100** (Colab Pro)
3. Click **Save**

| GPU | VRAM | Recommended Model | Speed |
|-----|------|-------------------|-------|
| T4 | 16 GB | `llama3.1:8b` | Medium |
| L4 | 24 GB | `qwen2.5:14b` | Fast |
| A100 | 40 GB | `mixtral:8x7b` | Very Fast |

In [ ]:
#@title 1. Check GPU
!nvidia-smi --query-gpu=name,memory.total --format=csv
import torch
print(f"\nCUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"VRAM: {vram_gb:.1f} GB")
    if vram_gb >= 16:
        print("\nRecommended: llama3.1:8b or qwen2.5:7b")
    elif vram_gb >= 24:
        print("\nRecommended: qwen2.5:14b or mistral-nemo:12b")
else:
    print("\nNo GPU! Please enable GPU runtime.")

---
# Option A: Ollama (FREE, Local)
Run the following cells to use Ollama (no API key needed)

In [ ]:
#@title 2A. Install Ollama
!curl -fsSL https://ollama.ai/install.sh | sh
print("\nOllama installed!")

In [ ]:
#@title 3A. Start Ollama Server
import subprocess
import time
import requests

subprocess.Popen(["ollama", "serve"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
print("Waiting for Ollama server...")
time.sleep(5)

try:
    r = requests.get("http://localhost:11434/api/tags", timeout=5)
    if r.status_code == 200:
        print("Ollama server running!")
except:
    print("Ollama server not reachable. Please run this cell again.")

In [ ]:
#@title 6. PDF Translator Code
import requests
import re
from dataclasses import dataclass
from typing import List, Tuple, Optional

@dataclass
class Block:
    page: int
    content: str
    translated: str = ""

OLLAMA_URL = "http://localhost:11434"

# Language codes
LANGUAGES = {
    "German": "de", "English": "en", "French": "fr", "Spanish": "es",
    "Italian": "it", "Japanese": "ja", "Chinese": "zh", "Portuguese": "pt",
    "Russian": "ru", "Korean": "ko", "Arabic": "ar", "Ukrainian": "uk",
    "Hebrew": "he", "Dutch": "nl", "Polish": "pl", "Turkish": "tr",
    "Swedish": "sv", "Czech": "cs", "Greek": "el", "Hindi": "hi"
}

def protect_formulas(text: str) -> tuple:
    """Protect math formulas from translation."""
    protected = {}
    counter = [0]
    def protect(match):
        key = f"__FORMULA_{counter[0]}__"
        protected[key] = match.group(0)
        counter[0] += 1
        return key
    patterns = [
        r'\$\$[\s\S]*?\$\$',
        r'\$[^$\n]+\$',
        r'\\begin\{[^}]+\}[\s\S]*?\\end\{[^}]+\}',
        r'\\[a-zA-Z]+\{[^}]*\}',
        r'\\[a-zA-Z]+',
        r'\d+\.?\d*\s*[×x]\s*10\^[−\-]?\d+',
    ]
    result = text
    for pattern in patterns:
        result = re.sub(pattern, protect, result)
    return result, protected

def restore_formulas(text: str, protected: dict) -> str:
    """Restore protected formulas after translation."""
    for key, value in protected.items():
        text = text.replace(key, value)
    return text

def extract_pdf(pdf_path: str) -> Tuple[List[Block], str]:
    import pdfplumber
    from langdetect import detect
    blocks = []
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text(layout=True) or ""
            for para in text.split("\n\n"):
                para = para.strip()
                if para:
                    blocks.append(Block(page=i+1, content=para))
    sample = " ".join(b.content for b in blocks[:3])
    lang = detect(sample) if sample else "en"
    return blocks, lang

def translate_ollama(text: str, model: str, source: str, target: str) -> str:
    if not text.strip():
        return text
    # Protect formulas
    protected_text, formulas = protect_formulas(text)
    system = f"""You are a professional scientific translator.
Translate ALL text to {target} ONLY.
Keep all __FORMULA_X__ placeholders unchanged.
Output ONLY the translation."""
    user = f"Translate from {source} to {target}:\n\n{protected_text}"
    try:
        r = requests.post(f"{OLLAMA_URL}/api/chat", json={
            "model": model,
            "messages": [{"role": "system", "content": system}, {"role": "user", "content": user}],
            "stream": False,
            "options": {"temperature": 0.1, "num_predict": 8192}
        }, timeout=300)
        if r.status_code == 200:
            result = r.json().get("message", {}).get("content", protected_text)
            return restore_formulas(result, formulas)
    except Exception as e:
        print(f"Ollama error: {e}")
    return text

def translate_openai(text: str, api_key: str, source: str, target: str) -> str:
    if not text.strip() or not api_key:
        return text
    # Protect formulas
    protected_text, formulas = protect_formulas(text)
    from openai import OpenAI
    client = OpenAI(api_key=api_key)
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": f"Translate to {target}. Keep __FORMULA_X__ placeholders unchanged. Output only translation."},
                {"role": "user", "content": protected_text}
            ],
            temperature=0.1
        )
        result = response.choices[0].message.content
        return restore_formulas(result, formulas)
    except Exception as e:
        print(f"OpenAI error: {e}")
    return text

def translate_pdf(pdf_path: str, target: str, backend: str = "ollama", model: str = "llama3.1:8b", api_key: str = None) -> str:
    print(f"📄 Extracting: {pdf_path}")
    blocks, source = extract_pdf(pdf_path)
    print(f"📊 {len(blocks)} blocks, detected language: {source}")
    print(f"🌍 Translating to {target} with {backend}...")
    for i, b in enumerate(blocks):
        print(f"  ⏳ Block {i+1}/{len(blocks)}", end="\r")
        if backend == "openai" and api_key:
            b.translated = translate_openai(b.content, api_key, source, target)
        else:
            b.translated = translate_ollama(b.content, model, source, target)
    print(f"\n✅ Translation complete!")
    return "\n\n".join(b.translated for b in blocks)

print("✅ PDF Translator ready!")

#@title 7. Upload and Translate PDF
from google.colab import files

#@markdown ### Settings
target_language = "German" #@param ["German", "English", "French", "Spanish", "Italian", "Japanese", "Chinese", "Portuguese", "Russian", "Korean", "Arabic", "Ukrainian", "Hebrew", "Dutch", "Polish", "Turkish", "Swedish", "Czech", "Greek", "Hindi"]
backend = "ollama" #@param ["ollama", "openai"]
ollama_model = "llama3.1:8b" #@param ["llama3.1:8b", "qwen2.5:7b", "qwen2.5:14b", "mistral:7b", "mistral-nemo:12b"]

# Get OpenAI key if using OpenAI
api_key = None
if backend == "openai":
    try:
        from google.colab import userdata
        api_key = userdata.get('OPENAI_API_KEY')
        if not api_key:
            print("⚠️ No OpenAI API key found! Add it to Colab Secrets (key icon in sidebar).")
    except:
        print("⚠️ Could not access Colab Secrets.")

print("📤 Upload your PDF file...")
uploaded = files.upload()

for filename in uploaded.keys():
    print(f"\n📄 Processing: {filename}")
    result = translate_pdf(filename, target_language, backend, ollama_model, api_key)
    output_file = f"translated_{filename.replace('.pdf', '.txt')}"
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(result)
    print(f"\n✅ Translation complete!")
    print(f"📥 Downloading: {output_file}")
    files.download(output_file)

In [ ]:
#@title 2B. Setup OpenAI (Optional)
!pip install -q openai

# Try to get API key from Colab Secrets
try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    if OPENAI_API_KEY:
        print("OpenAI API key loaded from Colab Secrets!")
        print("Your key is secure and only visible to you.")
    else:
        print("No API key found. Add it to Colab Secrets (key icon in sidebar).")
except:
    print("Colab Secrets not available. You can enter key manually below.")
    OPENAI_API_KEY = None

In [ ]:
#@title 5. Install Dependencies
!pip install -q PyPDF2 pdfplumber langdetect requests
print("Dependencies installed!")

In [ ]:
#@title 6. PDF Translator Code
import requests
from dataclasses import dataclass
from typing import List, Tuple, Optional

@dataclass
class Block:
    page: int
    content: str
    translated: str = ""

OLLAMA_URL = "http://localhost:11434"

def extract_pdf(pdf_path: str) -> Tuple[List[Block], str]:
    import pdfplumber
    from langdetect import detect
    blocks = []
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text(layout=True) or ""
            for para in text.split("\n\n"):
                para = para.strip()
                if para:
                    blocks.append(Block(page=i+1, content=para))
    sample = " ".join(b.content for b in blocks[:3])
    lang = detect(sample) if sample else "en"
    return blocks, lang

def translate_ollama(text: str, model: str, source: str, target: str) -> str:
    if not text.strip():
        return text
    system = f"""You are a professional scientific translator.
You MUST translate ALL text to {target} ONLY.
NEVER use any other language than {target}.
Keep all mathematical formulas unchanged.
Output ONLY the translation."""
    user = f"Translate from {source} to {target}. Keep formulas unchanged.\n\nText:\n{text}"
    try:
        r = requests.post(f"{OLLAMA_URL}/api/chat", json={
            "model": model,
            "messages": [{"role": "system", "content": system}, {"role": "user", "content": user}],
            "stream": False,
            "options": {"temperature": 0.1, "num_predict": 8192}
        }, timeout=300)
        if r.status_code == 200:
            return r.json().get("message", {}).get("content", text)
    except Exception as e:
        print(f"Ollama error: {e}")
    return text

def translate_openai(text: str, api_key: str, source: str, target: str) -> str:
    if not text.strip() or not api_key:
        return text
    from openai import OpenAI
    client = OpenAI(api_key=api_key)
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": f"Translate to {target}. Keep formulas unchanged. Output only translation."},
                {"role": "user", "content": text}
            ],
            temperature=0.1
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"OpenAI error: {e}")
    return text

def translate_pdf(pdf_path: str, target: str, backend: str = "ollama", model: str = "llama3.1:8b", api_key: str = None) -> str:
    print(f"Extracting: {pdf_path}")
    blocks, source = extract_pdf(pdf_path)
    print(f"{len(blocks)} blocks, language: {source}")
    print(f"Translating to {target} with {backend}...")
    for i, b in enumerate(blocks):
        print(f"  Block {i+1}/{len(blocks)}", end="\r")
        if backend == "openai" and api_key:
            b.translated = translate_openai(b.content, api_key, source, target)
        else:
            b.translated = translate_ollama(b.content, model, source, target)
    print("\nDone!")
    return "\n\n".join(b.translated for b in blocks)

print("PDF Translator ready!")

In [ ]:
#@title 7. Upload and Translate PDF
from google.colab import files

#@markdown ### Settings
target_language = "German" #@param ["German", "English", "French", "Spanish", "Italian", "Japanese", "Chinese"]
backend = "ollama" #@param ["ollama", "openai"]
ollama_model = "llama3.1:8b" #@param ["llama3.1:8b", "qwen2.5:7b", "qwen2.5:14b", "mistral:7b"]

# Get OpenAI key if using OpenAI
api_key = None
if backend == "openai":
    try:
        from google.colab import userdata
        api_key = userdata.get('OPENAI_API_KEY')
        if not api_key:
            print("No OpenAI API key found! Add it to Colab Secrets.")
    except:
        print("Could not access Colab Secrets.")

print("Upload PDF file...")
uploaded = files.upload()

for filename in uploaded.keys():
    print(f"\nProcessing: {filename}")
    result = translate_pdf(filename, target_language, backend, ollama_model, api_key)
    output_file = f"translated_{filename.replace('.pdf', '.txt')}"
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(result)
    print(f"\nDownloading: {output_file}")
    files.download(output_file)